In [162]:
!pip install plotly

In [163]:
import plotly.express as px

In [164]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import style
plt.style.use(['dark_background'])

sns.set(color_codes=True)
import urllib.request
import json

In [165]:
url = "https://api.covid19india.org/states_daily.json"
urllib.request.urlretrieve(url, 'data.json');

In [166]:
with open("data.json") as f:
    data = json.load(f)
data = data['states_daily']
df = pd.json_normalize(data)

In [167]:
df_pie = df[['status', 'tt']] 

In [168]:
df_pie['tt'] = pd.to_numeric(df_pie['tt'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [169]:
df_g = df_pie.groupby('status', as_index = False).sum()

In [170]:
df_g

,status,tt
0,Confirmed,4019555
1,Deceased,69646
2,Recovered,3104505


In [171]:
df_g.loc[0, ('tt')] -= df_g.loc[1, ('tt')]
df_g.loc[0, ('tt')] -= df_g.loc[2, ('tt')]

In [172]:
df_g.loc[0, 'status'] = 'Active'

In [176]:
import plotly.graph_objects as go
# colors = ['gold', 'mediumturquoise', 'darkorange']
colors = []
colors.append(px.colors.qualitative.G10[2])
colors.append(px.colors.qualitative.G10[1])
colors.append(px.colors.qualitative.G10[3])

fig = go.Figure(data=[go.Pie(labels=['Active', 'Deceased', 'Recovered'],
                             values=df_g['tt'], pull=[0.1, 0, 0], rotation = 180,
                             title = {'text' : 'Recovery Rate India',  'position' : 'bottom right',
                                      'font': {'size' : 20}},
                             showlegend = False)])
fig.update_traces(hoverinfo='label+value', textinfo='label+percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=1)))
fig.update_layout(
    autosize=False,
    width=768,
    height=505,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)
fig.show()

In [174]:
import plotly.io as pio
pio.write_html(fig, file='index.html', auto_open=True)

In [252]:
df_ = df.copy(deep = True)
df_['date'] = pd.to_datetime(df_['date'])
df_ = df_[['date', 'status', 'tt']]
df_['Cases'] = pd.to_numeric(df_['tt'])

In [253]:
df_conf = df_[df_['status'] == 'Confirmed']
df_rec  = df_[df_['status'] == 'Recovered']
df_dec  = df_[df_['status'] == 'Deceased']

In [254]:
colors = []
colors.append(px.colors.qualitative.G10[2])
colors.append(px.colors.qualitative.G10[1])
colors.append(px.colors.qualitative.G10[3])

In [255]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= df_conf['date'], y = df_conf['tt'], 
                         mode='markers',
                         marker=dict(
                         color=colors[0],
                         line=dict(
                            color='black',
                            width=0.2
                         )),
                         name="Confirmed", line_shape='spline',))
fig.add_trace(go.Scatter(x= df_rec['date'], y = df_rec['tt'], 
                         mode='markers',
                         marker=dict(
                         color=colors[2],
                         line=dict(
                            color='black',
                            width=0.2
                         )),
                         name="Recovered", line_shape='spline'))
fig.add_trace(go.Scatter(x= df_dec['date'], y = df_dec['tt'],
                         mode='markers',
                         marker=dict(
                         color=colors[1],
                         line=dict(
                            color='black',
                            width=0.2
                         )), 
                         name="Deceased", line_shape='spline'))

fig.update_traces(hoverinfo='text+y', mode='lines+markers')
fig.update_layout(
    xaxis_title = 'Months',
    yaxis_title = 'Cases',
    title = {
       'text': "Overall Cases in India",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top' 
    },
    legend=dict(y=0.5, traceorder='reversed', font_size=16))

fig.show()

In [256]:
import plotly.io as pio
pio.write_html(fig, file='area.html', auto_open=True)